In [1]:
import sys
import os
from pathlib import Path

# importディレクトリの追加
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
print(sys.path)

# プロキシの設定
# os.environ['HTTP_PROXY'] = ''
# os.environ['HTTPS_PROXY'] = ''

%matplotlib inline

['/home/y-katayama/notebooks/dl_study/02_pytorch_tutorial', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/y-katayama/venv/pt1.7/lib/python3.8/site-packages']


In [2]:
!nvidia-smi

Fri Mar 24 16:13:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 40%   37C    P8    17W / 184W |     16MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
| 40%   

In [3]:
import torch
import numpy as np
from torch import nn
import torch.autograd.profiler as profiler

In [4]:
%matplotlib inline

# PyTorchモジュールのプロファイリング方法

- プロファイラはモデル内のパフォーマンスのボトルネックを特定するうえで役立つ

- 入力の線形変換
- 線形変換の結果を用いたマスクテンソルのインデックスの取得

`profiler.record_fcuntion("label")を使用し、各サブタスクを行うコードを分離してラベル付けされたコンテクストマネージャ内にラップする
プロファイラの出力ではサブタスクないのすべての処理の総合的なパフォーマンス指標が、対応するラベルの↓に表示される**

ランダム入力とマスクテンソル、そしてモデルを初期化します。
プロファイラーを実行する前に、正確なパフォーマンスのベンチマークを測定するためにCUDAの準備をします。

- profiler.profileコンテクストマネージャー内でモジュールのフォワードパスをラップします。
- with_stack=Trueパラメーターは、<u>ファイルとトレース内の処理の行数を追加で記録する</u>
    - **with_stack=Trueはオーバーヘッドを生じる**ためコードを調査する目的のときのみ使用する。
    - **パフォーマンスを測定する際は、with_stack=Falseにすることを忘れないこと**

In [5]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        
    def forward(self, input, mask):
        with profiler.record_function('LINEAR PASS'):
            out = self.linear(input)
        
        with profiler.record_function('MASK INDICES'):
            threshold = out.sum(axis=1).mean().item()
            hi_idx = np.argwhere(mask.cpu().numpy() > threshold)
            hi_idx = torch.from_numpy(hi_idx).cuda()
        
        return out, hi_idx

# プロファイル結果の出力

- `profiler.key_averages`で処理名、任意で入力の形状、またはスタックトレースのイベント毎で結果を集約します。

- 入力の形状でグループ化を行うことは、どのテンソルの形状がモデルに利用されているか特定する上で役に立ちます。
- ここで、処理と（直近5つのイベントのみに切り取られた）処理のトレースバック毎に実行時間を集約するgroup_by_stack_n=5 を使用し、それらが登録された順番でイベントを表示します。
- 表は、sort_by引数を与えることでソートできます。有効なソートキーについては、ドキュメントをご参照ください。

- ノートブック内でプロファイラーを実行していると、スタックトレース内のファイル名ではなく、<ipython-input-18-193a910735e8>(13): forwardのようなエントリを目にするかもしれません。 これらは、<notebook-cell>(line number): calling-functionに対応しています。

In [6]:
model = MyModule(500, 10).cuda()
input = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.double).cuda()

# warm-up
model(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(input, mask)
    
# プロファイル結果の出力
print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                 MASK INDICES        70.92%        1.363s        99.97%        1.921s        1.921s          -4 b    -953.67 Mb       2.79 Gb      -1.00 Kb             1  /home/y-katayama/venv/pt1.7/lib/python3.8/site-packages/torch/autograd/prof  
   

# メモリパフォーマンスの向上

- 上記のプロファイル結果から、最も高コストな処理はマスクインデックスの処理`forward(10)`であることがわかる
- まずはメモリ消費の改善に取り組むことにする

- L.12の`.to().cpu()`で953.67Mbのメモリを消費していることがわかる`Self CPU Memを参照`
  - この処理はmaskをcpuにコピーしている処理
- torch.doubleをtorch.floatにキャストするとメモリ不可を軽減できるだろうか？
  - 以下、対象の処理ではCPUのメモリ使用量が半減することが確認できる

In [7]:
model = MyModule(500, 10).cuda()
input = torch.rand(128, 500).cuda()

# maskのデータ型をtorch.floatに変更
mask = torch.rand((500, 500, 500), dtype=torch.float).cuda()

# 準備
model(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(input, mask)

# 彩度プロファイルを取る
print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                 MASK INDICES        82.05%        1.276s        99.96%        1.554s        1.554s          -4 b    -476.84 Mb       1.40 Gb      -1.00 Kb             1  /home/y-katayama/venv/pt1.7/lib/python3.8/site-packages/torch/autograd/prof  
   

# 時間パフォーマンスの向上

- メモリ消費量に加えて。時間のコストもやや減らせましたが、依然として実行に時間が掛かり過ぎています。
- 行列をCUDAからCPUにコピーする処理がいかに高コストな処理であるかわかります。

- `forward (12)`内の`aten::copy_`は、`mask`をCPUにコピーし、NumPyの`argwhere`関数を使用できるようにしています。
- そして`forward(13)`の`aten::copy_`は、配列をテンソルとしてCUDAにコピーし直しています。
- 代わりに`torch.nonzero()`を使用すれば、これら2つの処理を両方取り除くことができます。

In [8]:
class MyModule_reduce_time(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule_reduce_time, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)

    def forward(self, input, mask):
        with profiler.record_function("LINEAR PASS"):
            out = self.linear(input)

        with profiler.record_function("MASK INDICES"):
            threshold = out.sum(axis=1).mean()
            # hi_idx = np.argwhere(mask.cpu().numpy() > threshold)
            # マスクと閾値の比較処理をnonzero()をtorch.nn.nonzero()使って判定するように変更
            hi_idx = (mask > threshold).nonzero(as_tuple=True)

        return out, hi_idx

In [9]:
model_2 = MyModule_reduce_time(500, 10).cuda()
input = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.float).cuda()

# warm-up
model_2(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model_2(input, mask)

print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
          aten::nonzero        87.42%       8.154ms        88.81%       8.284ms       8.284ms           0 b           0 b     668.97 Mb           0 b             1  /tmp/ipykernel_168484/3383356899.py(14): forward                             
                           

- Numpyとの変換処理取り除かれたことで、推論時間が激減することが確認できた。(Self CPU time total: 1.555s -> Self CPU time total: 9.328ms)